In [29]:
from project import *
import csv
import mysql.connector as sql
from mysql.connector import Error

In [105]:
clear_database(cursor, dbConn, tables)

In [106]:
# get connection 
# dbConn = create_connection("localhost", "root", "123456", "progDB")
# cursor = dbConn.cursor()
# create_database(cursor, "progDB")
create_tables_from_file_m(cursor, "test_schema.sql", dbConn)

In [32]:
# create the tables from file
# NOTE: need to rework for the if not exists  
def create_tables_from_file_m(cursor, filename, connection):
    try:
        with open(filename,'r') as file:
            sql_script = file.read()
            
        for result in cursor.execute(sql_script, multi=True):
            pass
    except Error as e:
        print(f"The error '{e} occurred'")
    
    connection.commit()
    

## General Database Management

In [98]:
tables = ["Programs", "Departments", "Faculty", "Courses",
          "Semesters", "CourseSections", "LearningObjectives", 
          "ProgramObjectives", "SectionObjectives", "SubObjectives",
          "ProgramCourses"]

def clear_database(cursor, connection, tableList):
    try:
        cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
        for table in tableList:
            sqlcom = "DROP TABLE IF EXISTS %s" % (table)
            cursor.execute(sqlcom)
        cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
    except Error as e:
        print(f"The error '{e} occurred'")
    connection.commit()

In [83]:
def check_dept_exists(dept_code):
    try:
        com = "SELECT * FROM Departments WHERE DepartmentCode = '%s'" % (dept_code) 
        cursor.execute(com)
        
        if cursor.fetchall():
            return True
        
        print("Department does not exist. Please check department code.")
        return False
    except Error as e:
        print(f"Error executing Department Select: {e}")
        
        
def check_fal_exists(fal_id):
    try:
        com = "SELECT * FROM Faculty WHERE FacultyID = '%s'" % (fal_id) 
        cursor.execute(com)
        
        if cursor.fetchall():
            return True
        
        print("Faculty member does not exist. Please check faculty id")
        return False
    except Error as e:
        print(f"Error executing Faculty Select: {e}")
        
def check_program_exists(prog_name):
    try:
        com = "SELECT * FROM Programs WHERE ProgramName = '%s'" % (prog_name) 
        cursor.execute(com)
        
        if cursor.fetchall():
            return True
        
        print("Program does not exist. Please check program name")
        return False
    except Error as e:
        print(f"Error executing Program Select: {e}")
    
def enter_program_info(cursor, connection, prog_name, pc_id, dept_code):
    '''Check to see if program coordinator exists, if not add them to the faculty table'''
    
    #check if department exists, if not, return error 
    if not check_dept_exists(dept_code):
        return False
    
    if not check_fal_exists(pc_id):
        return False
    
    try:
        # Insert data into the Programs table
        sqlcom = "INSERT INTO Programs \
        (ProgramName, ProgramCoordinatorID, DepartmentCode) \
        VALUES (%s, %s, %s)"
        
        program_data = (prog_name, pc_id, dept_code)
        
        cursor.execute(sqlcom, program_data)
        # Commit the transaction
        connection.commit()

        print(f"Data for program '{prog_name}' successfully inserted into the Programs table.")
    except Error as e:
        print(f"Error inserting program data: {e}")
    

def enter_department_info(cursor, connection, dept_code, dept_name):
    try:
        # Insert data into the Departments table
        sqlcom = "INSERT INTO Departments (DepartmentCode, DepartmentName) \
            VALUES (%s, %s)"
        dept_info = (dept_code, dept_name)
        cursor.execute(sqlcom, dept_info)

        # Commit the transaction
        connection.commit()

        print(f"Data for department '{dept_name}' successfully inserted into the Departments table.")
    except Error as e:
        print(f"Error inserting department data: {e}")

def enter_faculty_info(cursor, connection, fac_id, fac_name, fac_email, dept_code, fac_rank=None):
    try:
        if not check_dept_exists(dept_code):
            return
    
        # Insert data into the Faculty table
        sqlcom = "INSERT INTO Faculty (FacultyID, FacultyName, FacultyEmail, FacultyRank, DepartmentCode) \
        VALUES (%s, %s, %s, %s, %s)"
        faculty_data = (fac_id, fac_name, fac_email, fac_rank, dept_code)
        cursor.execute(sqlcom, faculty_data)

        # Commit the transaction
        connection.commit()

        print(f"Data for faculty '{fac_name}' successfully inserted into the Faculty table.")
    except Error as e:
        print(f"Error inserting faculty data: {e}")

def enter_course_info(cursor, connection, co_id, co_title, co_desc, dept_code):
    try:
        # Insert data into the Courses table
        sqlcom = "INSERT INTO Courses (CourseID, CourseTitle, CourseDescription, DepartmentCode) \
            VALUES (%s, %s, %s, %s)"
        course_data = (co_id, co_title, co_desc, dept_code)
        cursor.execute(sqlcom, course_data)

        # Commit the transaction
        connection.commit()

        print(f"Data for course '{co_title}' successfully inserted into the Courses table.")
    except Error as e:
        print(f"Error inserting course data: {e}")
    

In [108]:
enter_department_info(cursor, dbConn, "0000", "Test Department")
#enter_faculty_info(cursor, dbConn, "1111111", "Test Falc", "faltest@gmail.com", "0000")
#enter_program_info(cursor, dbConn, "Test 2 Program", "1111112", "0000")
enter_faculty_info(cursor, dbConn, "1111113", "Test Falc 3", "faltest3@gmail.com", "0000", "boop")

Data for department 'Test Department' successfully inserted into the Departments table.
Error inserting faculty data: 3819 (HY000): Check constraint 'faculty_chk_1' is violated.


## User Input Handling
- check emails
- delete all 